In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch.optim import Adam
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchaudio
import IPython.display as ipd
import timm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import os
import gc
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import folium
from folium.plugins import HeatMap, MarkerCluster
import webbrowser

In [ ]:
df = pd.read_csv("../data/train_metadata.csv")

In [ ]:
df.head()

<h3> Creating barplots <h3>

In [ ]:
plt.figure(figsize=(24, 8))
most_common_names = df["common_name"].value_counts().nlargest(40).index
barplot_data = df[df["common_name"].isin(most_common_names)]
common_name_counts = barplot_data["common_name"].value_counts().sort_values(ascending=False)

sns.barplot(x=common_name_counts.index, y=common_name_counts.values, palette="magma")
plt.xticks(rotation=45, ha="right")
plt.title("Distribution of the species of most frequent birds", fontsize=16)
plt.xlabel("Name", fontsize=14)
plt.ylabel('Count', fontsize=14)

In [ ]:
plt.figure(figsize=(24, 8))
least_common_names = df["common_name"].value_counts().nsmallest(40).index
barplot_data = df[df["common_name"].isin(least_common_names)]
common_name_counts = barplot_data["common_name"].value_counts().sort_values(ascending=False)

sns.barplot(x=common_name_counts.index, y=common_name_counts.values, palette="magma")
plt.xticks(rotation=45, ha="right")
plt.title("Distribution of the species of least frequent birds", fontsize=16)
plt.xlabel("Name", fontsize=14)
plt.ylabel("Count", fontsize=14)

<h3> Creating heatmap and pointer map <h3>

In [ ]:
map_data = df.dropna(subset=["latitude", "longitude"])
center_lat = map_data["latitude"].mean()
center_lon = map_data["longitude"].mean()

In [ ]:
my_map = folium.Map(location=[center_lat, center_lon], zoom_start=2)
heat_data = [[row["latitude"], row["longitude"]] for index, row in map_data.iterrows()]
heatmap = HeatMap(heat_data, radius=8, blur=0.5, max_zoom=0, opacity=0.3).add_to(my_map)
heatmap.save("../plots/birds_heatmap.html")

In [ ]:
my_map_marker = folium.Map(location=[center_lat, center_lon], zoom_start=2)
for index, row in map_data.iloc[::4].iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=2,
        color="purple",
        fill=True,
        fill_color="purple",
        fill_opacity=0.7,
    ).add_to(my_map_marker)
my_map_marker.save("../plots/birds_markers.html")